In [1]:
import pandas as pd
import numpy as np 
import glob
from biopandas.pdb import PandasPdb
import warnings
warnings.filterwarnings("ignore")
from get_distances import *
import sys
from multiprocessing import Pool
import time
import math

In [38]:
phosphosite_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/phosphosite_for_pockets.csv", nrows = 10000)
stability_data = pd.read_csv("/rcfs/projects/proteometer/ProtVar/predictions/stability/2024.05.28_foldx_energy.csv") # the stability dataset is BIG. It has 208792558 rows.  Just loading in the first 100,000
stability_data

# note that the .sample() still reads in the whole csv and just samples from it.  Better to use the nrows so it just reads the first n rows

,protein_acc,wild_type,position,mutated_type,foldx_ddg,plddt
0,A0A075B6S0,N,1,A,-0.001835,56.90
1,A0A075B6S0,N,1,C,-0.011188,56.90
2,A0A075B6S0,N,1,D,-0.041779,56.90
3,A0A075B6S0,N,1,E,-0.024084,56.90
4,A0A075B6S0,N,1,F,0.324027,56.90
...,...,...,...,...,...,...
208792553,Q9P273,R,2699,S,0.620197,42.24
208792554,Q9P273,R,2699,T,0.650257,42.24
208792555,Q9P273,R,2699,V,0.809246,42.24
208792556,Q9P273,R,2699,W,0.276514,42.24


In [3]:
phosphosite_data

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position,uniprot_id
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,0,2,P31946_2,P31946,THR,2.0,NaN,undefined,2.0,P31946
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,0,6,P31946_6,P31946,SER,6.0,NaN,undefined,6.0,P31946
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,0,21,P31946_21,P31946,TYR,21.0,11.100927,protonated,21.0,P31946
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,0,32,P31946_32,P31946,THR,32.0,NaN,undefined,32.0,P31946
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,0,39,P31946_39,P31946,SER,39.0,NaN,undefined,39.0,P31946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,AKAP8L,AKAP8L,Q9ULX6,19p13.12,S326-p,33526845,human,71.64,NaN,...,0,326,Q9ULX6_326,Q9ULX6,SER,326.0,NaN,undefined,326.0,Q9ULX6
9996,9996,AKAP8L,AKAP8L,Q9ULX6,19p13.12,Y435-p,40247902,human,71.64,AKAP95,...,0,435,Q9ULX6_435,Q9ULX6,TYR,435.0,9.263724,protonated,435.0,Q9ULX6
9997,9997,AKAP8L,AKAP8L,Q9ULX6,19p13.12,T504-p,25294382,human,71.64,AKAP95,...,0,504,Q9ULX6_504,Q9ULX6,THR,504.0,NaN,undefined,504.0,Q9ULX6
9998,9998,AKAP8L,AKAP8L,Q9ULX6,19p13.12,T550-p,36136278,human,71.64,NaN,...,0,550,Q9ULX6_550,Q9ULX6,THR,550.0,NaN,undefined,550.0,Q9ULX6


In [4]:
# adding empty columns to the dataset (all nan because expected to be float)

phosphosite_data['plddt'] = np.nan
phosphosite_data['min_foldx'] = np.nan
phosphosite_data['max_foldx'] = np.nan
phosphosite_data['absval_change_foldx'] = np.nan
phosphosite_data

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,AA,res_number,pK,state,position,uniprot_id,plddt,min_foldx,max_foldx,absval_change_foldx
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,THR,2.0,NaN,undefined,2.0,P31946,NaN,NaN,NaN,NaN
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,SER,6.0,NaN,undefined,6.0,P31946,NaN,NaN,NaN,NaN
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,TYR,21.0,11.100927,protonated,21.0,P31946,NaN,NaN,NaN,NaN
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,THR,32.0,NaN,undefined,32.0,P31946,NaN,NaN,NaN,NaN
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,SER,39.0,NaN,undefined,39.0,P31946,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,AKAP8L,AKAP8L,Q9ULX6,19p13.12,S326-p,33526845,human,71.64,NaN,...,SER,326.0,NaN,undefined,326.0,Q9ULX6,NaN,NaN,NaN,NaN
9996,9996,AKAP8L,AKAP8L,Q9ULX6,19p13.12,Y435-p,40247902,human,71.64,AKAP95,...,TYR,435.0,9.263724,protonated,435.0,Q9ULX6,NaN,NaN,NaN,NaN
9997,9997,AKAP8L,AKAP8L,Q9ULX6,19p13.12,T504-p,25294382,human,71.64,AKAP95,...,THR,504.0,NaN,undefined,504.0,Q9ULX6,NaN,NaN,NaN,NaN
9998,9998,AKAP8L,AKAP8L,Q9ULX6,19p13.12,T550-p,36136278,human,71.64,NaN,...,THR,550.0,NaN,undefined,550.0,Q9ULX6,NaN,NaN,NaN,NaN


In [5]:
# changing stability position data to be float so it matches phosphosite
stability_data['position_float'] = stability_data['position'].astype(float)
stability_data

,protein_acc,wild_type,position,mutated_type,foldx_ddg,plddt,position_float
0,A0A075B6S0,N,1,A,-0.001835,56.90,1.0
1,A0A075B6S0,N,1,C,-0.011188,56.90,1.0
2,A0A075B6S0,N,1,D,-0.041779,56.90,1.0
3,A0A075B6S0,N,1,E,-0.024084,56.90,1.0
4,A0A075B6S0,N,1,F,0.324027,56.90,1.0
...,...,...,...,...,...,...,...
9999995,A6NNP5,R,61,M,-0.305345,95.54,61.0
9999996,A6NNP5,R,61,N,-0.196676,95.54,61.0
9999997,A6NNP5,R,61,P,3.421740,95.54,61.0
9999998,A6NNP5,R,61,Q,0.484374,95.54,61.0


In [8]:
test_stability = stability_data.query('position_float == 1.0 and protein_acc == "A6NNP5"')
test_stability

,protein_acc,wild_type,position,mutated_type,foldx_ddg,plddt,position_float
9998845,A6NNP5,M,1,A,0.002684,46.97,1.0
9998846,A6NNP5,M,1,C,0.050966,46.97,1.0
9998847,A6NNP5,M,1,D,0.076688,46.97,1.0
9998848,A6NNP5,M,1,E,0.043943,46.97,1.0
9998849,A6NNP5,M,1,F,-0.027688,46.97,1.0
9998850,A6NNP5,M,1,G,-0.030278,46.97,1.0
9998851,A6NNP5,M,1,H,0.102543,46.97,1.0
9998852,A6NNP5,M,1,I,0.176817,46.97,1.0
9998853,A6NNP5,M,1,K,0.001120,46.97,1.0
9998854,A6NNP5,M,1,L,0.055148,46.97,1.0


In [10]:
1 in test_stability['position'].unique()

True

In [13]:
test_stability['foldx_ddg'].min()

np.float64(-0.184459)

In [14]:
test_stability['foldx_ddg'].max()

np.float64(0.176817)

In [15]:
test_stability['foldx_ddg'].abs().max()

np.float64(0.184459)

In [16]:
test_stability['foldx_ddg'].abs().median()

np.float64(0.0526825)

In [68]:
stability_data[stability_data['protein_acc'] == "A6NNP5" & stability_only_uniprot['position_float'] == 1.0]

NameError: name 'stability_only_uniprot' is not defined

In [80]:
unique_uniprot_psp[0]

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,AA,res_number,pK,state,position,uniprot_id,plddt,min_foldx,max_foldx,absval_change_foldx
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,THR,2.0,NaN,undefined,2.0,P31946,64.09,NaN,NaN,NaN
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,SER,6.0,NaN,undefined,6.0,P31946,74.95,NaN,NaN,NaN
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,TYR,21.0,11.100927,protonated,21.0,P31946,88.51,NaN,NaN,NaN
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,THR,32.0,NaN,undefined,32.0,P31946,58.79,NaN,NaN,NaN
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,SER,39.0,NaN,undefined,39.0,P31946,48.80,NaN,NaN,NaN
5,5,YWHAB,14-3-3 beta,P31946,20q13.12,S47-p,4773271,human,28.08,14-3-3,...,SER,47.0,NaN,undefined,47.0,P31946,71.08,NaN,NaN,NaN
6,6,YWHAB,14-3-3 beta,P31946,20q13.12,Y50-p,12432961,human,28.08,14-3-3,...,TYR,50.0,NaN,undefined,50.0,P31946,56.13,NaN,NaN,NaN
7,7,YWHAB,14-3-3 beta,P31946,20q13.12,S59-p,4935095,human,28.08,14-3-3,...,SER,59.0,NaN,undefined,59.0,P31946,55.02,NaN,NaN,NaN
8,8,YWHAB,14-3-3 beta,P31946,20q13.12,S60-p,450226,human,28.08,14-3-3,...,SER,60.0,NaN,undefined,60.0,P31946,47.81,NaN,NaN,NaN
9,9,YWHAB,14-3-3 beta,P31946,20q13.12,S66-p,15718715,human,28.08,14-3-3,...,SER,66.0,NaN,undefined,66.0,P31946,48.24,NaN,NaN,NaN


In [87]:
# adding the plddt column 
unique_uniprot_psp = [phosphosite_data[phosphosite_data["uniprot_id"]==uniprot_id].copy() for uniprot_id in phosphosite_data["uniprot_id"].unique()]

for psp_only_uniprot in unique_uniprot_psp:
    uniprot = psp_only_uniprot["uniprot_id"].to_list()[0]
    print(uniprot)
    stability_only_uniprot = stability_data[stability_data['protein_acc'] == uniprot]
    for phosphosite_row_index in psp_only_uniprot.index:
        psp_res_number = psp_only_uniprot.loc[phosphosite_row_index,'res_number']
        #print(psp_res_number)
        stability_only_uniprot = stability_data[stability_data['position_float'] == psp_res_number]
        if not stability_only_uniprot.empty:
            plddt_to_add = stability_only_uniprot['plddt'].values[0]
            psp_only_uniprot.loc[phosphosite_row_index, 'plddt'] = plddt_to_add
            print("added", plddt_to_add)

pd.concat(unique_uniprot_psp)
#unique_uniprot_psp[0]

P31946
added 64.09
added 74.95
added 88.51
added 58.79
added 48.8
added 71.08
added 56.13
added 55.02
added 47.81
added 48.24
added 93.16
added 56.27
added 57.6
added 49.11
added 91.07
added 84.22
added 49.0
added 95.97
added 37.13
added 52.07
added 31.58
added 53.52
added 69.91
P62258
added 64.75
added 79.21
added 66.62
added 55.02
added 54.79
added 66.95
added 84.96
added 93.32
added 67.3
added 44.88
added 41.1
added 59.17
added 55.4
added 54.09
added 49.31
added 56.42
added 69.91
Q04917
added 89.15
added 61.39
added 64.75
added 79.21
added 66.62
added 55.02
added 66.95
added 36.85
added 37.05
added 91.07
added 41.1
added 50.35
added 58.26
added 68.6
added 52.78
P61981
added 89.15
added 72.81
added 64.75
added 79.21
added 66.62
added 55.02
added 66.95
added 58.74
added 56.98
added 67.3
added 36.85
added 91.07
added 41.1
added 71.54
added 50.35
added 68.6
added 38.41
added 52.78
added 62.26
added 37.13
added 81.48
added 56.42
P31947
added 74.29
added 92.34
added 40.02
added 67.05
adde

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,AA,res_number,pK,state,position,uniprot_id,plddt,min_foldx,max_foldx,absval_change_foldx
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,THR,2.0,NaN,undefined,2.0,P31946,64.09,NaN,NaN,NaN
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,SER,6.0,NaN,undefined,6.0,P31946,74.95,NaN,NaN,NaN
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,TYR,21.0,11.100927,protonated,21.0,P31946,88.51,NaN,NaN,NaN
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,THR,32.0,NaN,undefined,32.0,P31946,58.79,NaN,NaN,NaN
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,SER,39.0,NaN,undefined,39.0,P31946,48.80,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,AKAP8L,AKAP8L,Q9ULX6,19p13.12,S326-p,33526845,human,71.64,NaN,...,SER,326.0,NaN,undefined,326.0,Q9ULX6,NaN,NaN,NaN,NaN
9996,9996,AKAP8L,AKAP8L,Q9ULX6,19p13.12,Y435-p,40247902,human,71.64,AKAP95,...,TYR,435.0,9.263724,protonated,435.0,Q9ULX6,NaN,NaN,NaN,NaN
9997,9997,AKAP8L,AKAP8L,Q9ULX6,19p13.12,T504-p,25294382,human,71.64,AKAP95,...,THR,504.0,NaN,undefined,504.0,Q9ULX6,NaN,NaN,NaN,NaN
9998,9998,AKAP8L,AKAP8L,Q9ULX6,19p13.12,T550-p,36136278,human,71.64,NaN,...,THR,550.0,NaN,undefined,550.0,Q9ULX6,NaN,NaN,NaN,NaN


took 5 mins to complete 10,000 rows...

In [88]:
unique_uniprot_psp[0]

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,AA,res_number,pK,state,position,uniprot_id,plddt,min_foldx,max_foldx,absval_change_foldx
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,THR,2.0,NaN,undefined,2.0,P31946,64.09,NaN,NaN,NaN
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,SER,6.0,NaN,undefined,6.0,P31946,74.95,NaN,NaN,NaN
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,TYR,21.0,11.100927,protonated,21.0,P31946,88.51,NaN,NaN,NaN
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,THR,32.0,NaN,undefined,32.0,P31946,58.79,NaN,NaN,NaN
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,SER,39.0,NaN,undefined,39.0,P31946,48.80,NaN,NaN,NaN
5,5,YWHAB,14-3-3 beta,P31946,20q13.12,S47-p,4773271,human,28.08,14-3-3,...,SER,47.0,NaN,undefined,47.0,P31946,71.08,NaN,NaN,NaN
6,6,YWHAB,14-3-3 beta,P31946,20q13.12,Y50-p,12432961,human,28.08,14-3-3,...,TYR,50.0,NaN,undefined,50.0,P31946,56.13,NaN,NaN,NaN
7,7,YWHAB,14-3-3 beta,P31946,20q13.12,S59-p,4935095,human,28.08,14-3-3,...,SER,59.0,NaN,undefined,59.0,P31946,55.02,NaN,NaN,NaN
8,8,YWHAB,14-3-3 beta,P31946,20q13.12,S60-p,450226,human,28.08,14-3-3,...,SER,60.0,NaN,undefined,60.0,P31946,47.81,NaN,NaN,NaN
9,9,YWHAB,14-3-3 beta,P31946,20q13.12,S66-p,15718715,human,28.08,14-3-3,...,SER,66.0,NaN,undefined,66.0,P31946,48.24,NaN,NaN,NaN


In [39]:
phosphosite_data['plddt'] = np.nan
phosphosite_data['foldx_ddg_min'] = np.nan
phosphosite_data['foldx_ddg_max'] = np.nan
phosphosite_data['foldx_ddg_abs_max'] = np.nan
phosphosite_data['foldx_ddg_abs_median'] = np.nan



stability_data['position'] = stability_data['position'].astype(float)
unique_uniprot_psp = [phosphosite_data[phosphosite_data["uniprot_id"]==uniprot_id].copy() for uniprot_id in phosphosite_data.loc[:100,"uniprot_id"].unique()]
unique_uniprot_stability = [stability_data[stability_data["protein_acc"]==uniprot_id].copy() for uniprot_id in phosphosite_data.loc[:100,"uniprot_id"].unique()]
psp_and_stability = [[psp, stability] for (psp, stability) in zip(unique_uniprot_psp,unique_uniprot_stability)]

In [37]:
stability_data

,protein_acc,wild_type,position,mutated_type,foldx_ddg,plddt,position_float
0,A0A075B6S0,N,1.0,A,-0.001835,56.90,1.0
1,A0A075B6S0,N,1.0,C,-0.011188,56.90,1.0
2,A0A075B6S0,N,1.0,D,-0.041779,56.90,1.0
3,A0A075B6S0,N,1.0,E,-0.024084,56.90,1.0
4,A0A075B6S0,N,1.0,F,0.324027,56.90,1.0
...,...,...,...,...,...,...,...
9999995,A6NNP5,R,61.0,M,-0.305345,95.54,61.0
9999996,A6NNP5,R,61.0,N,-0.196676,95.54,61.0
9999997,A6NNP5,R,61.0,P,3.421740,95.54,61.0
9999998,A6NNP5,R,61.0,Q,0.484374,95.54,61.0


In [30]:
psp_and_stability = [[psp, stability] for (psp, stability) in zip(unique_uniprot_psp,unique_uniprot_stability)]

In [42]:
sys.getsizeof(stability_data)

42391207374

## Testing Parallelization

In [33]:
def add_plddt_and_foldx_ddg(psp_and_stability) -> pd.DataFrame:
    psp_only_uniprot, stability_only_uniprot = psp_and_stability
    uniprot = psp_only_uniprot["uniprot_id"].to_list()[0]
    print(uniprot)
    for phosphosite_row_index in psp_only_uniprot.index:
        psp_res_number = psp_only_uniprot.loc[phosphosite_row_index,'res_number']
        #print(psp_res_number)
        if psp_res_number in stability_only_uniprot['position'].unique():
            stability_only_res_uniprot = stability_only_uniprot[stability_only_uniprot['position'] == psp_res_number]
            plddt_to_add = stability_only_res_uniprot['plddt'].values[0]
            psp_only_uniprot.loc[phosphosite_row_index, 'plddt'] = plddt_to_add

            ### adding fold x data
            psp_only_uniprot.loc[phosphosite_row_index, 'foldx_ddg_min'] = stability_only_res_uniprot['foldx_ddg'].min()
            psp_only_uniprot.loc[phosphosite_row_index, 'foldx_ddg_max'] = stability_only_res_uniprot['foldx_ddg'].max()
            psp_only_uniprot.loc[phosphosite_row_index, 'foldx_ddg_abs_max'] = stability_only_res_uniprot['foldx_ddg'].abs().max()
            psp_only_uniprot.loc[phosphosite_row_index, 'foldx_ddg_abs_median'] = stability_only_res_uniprot['foldx_ddg'].abs().median()

    return(psp_only_uniprot)

In [34]:
start_time = time.perf_counter()
with Pool(4) as pool:
    output = pool.map(add_plddt_and_foldx_ddg, psp_and_stability)
finish_time = time.perf_counter()
full_df = pd.concat(output)

P62258Q04917P31946P61981





P31947


In [35]:
full_df

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,position,uniprot_id,plddt,min_foldx,max_foldx,absval_change_foldx,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
0,0,YWHAB,14-3-3 beta,P31946,20q13.12,T2-p,15718712,human,28.08,NaN,...,2.0,P31946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,YWHAB,14-3-3 beta,P31946,20q13.12,S6-p,15718709,human,28.08,NaN,...,6.0,P31946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,YWHAB,14-3-3 beta,P31946,20q13.12,Y21-p,3426383,human,28.08,14-3-3,...,21.0,P31946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,YWHAB,14-3-3 beta,P31946,20q13.12,T32-p,23077803,human,28.08,14-3-3,...,32.0,P31946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,YWHAB,14-3-3 beta,P31946,20q13.12,S39-p,27442700,human,28.08,14-3-3,...,39.0,P31946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,113,SFN,14-3-3 sigma,P31947,1p36.11,T207-p,56654093,human,27.77,14-3-3,...,207.0,P31947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,114,SFN,14-3-3 sigma,P31947,1p36.11,S209-p,55371904,human,27.77,14-3-3,...,209.0,P31947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,115,SFN,14-3-3 sigma,P31947,1p36.11,S216-p,455249,human,27.77,14-3-3,...,216.0,P31947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,116,SFN,14-3-3 sigma,P31947,1p36.11,T217-p,25239113,human,27.77,14-3-3,...,217.0,P31947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
stability_data[stability_data["protein_acc"]==uniprot_id]]